In [ ]:
import torch
import torch.nn as nn

In [ ]:
import torchvision
import torchvision.transforms as transfroms
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm

In [ ]:
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 10

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/Pytorch_Dataset/fruit_classification.zip', 'r') as zipObj:
    zipObj.extractall('fruit_classification')

In [ ]:
def get_dataframe(folder_path, state):
    class_name_list = os.listdir(folder_path)
    if state == 'train':
        file_lists = []
        for fruit in class_name_list:
            fruit_file_list = os.listdir(folder_path + '/' + fruit)
            file_lists.append([[folder_path + '/' + fruit + '/' + x, fruit] for x in fruit_file_list]) #list comprehension is faster in this case

        path_lists = []
        for list in file_lists:
            path_lists += list

        df = pd.DataFrame(path_lists, columns = ['path', 'label'])
    elif state == 'test':
        path_list = [folder_path + '/' + x for x in class_name_list]
        df = pd.DataFrame(path_list, columns = ['path'])

    return df


In [ ]:
df = get_dataframe('/content/fruit_classification/train/train', 'train')
CLASS_NUM = len(df['label'].unique())

In [ ]:
LabelEncoder = preprocessing.LabelEncoder()
df['label'] = LabelEncoder.fit_transform(df['label'].values)

In [ ]:
train_df, valid_df, _, _ = train_test_split(df, df['label'].values, test_size = 0.2, random_state = 10)

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, data_path, labels, transformer = None):
        self.path = data_path
        self.labels = labels
        self.transformer = transformer

    def __len__(self):
        return len(self.path)

    def __getitem__(self, idx):
        path = self.path[idx]
        image = cv.imread(path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        if self.transformer:
            image = self.transformer(image = image)['image']

        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        else:
            return image


In [ ]:
df_transformer = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), max_pixel_value = 255.0, always_apply = False, p=1.0),
    ToTensorV2()
])

In [ ]:
train_dataset = CustomDataset(train_df['path'].values, train_df['label'].values, df_transformer)
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0)

In [ ]:
valid_dataset = CustomDataset(valid_df['path'].values, valid_df['label'].values, df_transformer)
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0)

In [ ]:
class InvertedBlock(nn.Module):

    def __init__(self, input_channels, output_channels, strides = 1, padding = 1, bias = False, expansion=None):
        super().__init__()

        self.strides = strides
        self.same_channels = input_channels == output_channels

        if expansion is not None:
            hidden_channels = input_channels * expansion
        else:
            hidden_channels = input_channels

        self.convnet1 = nn.Sequential(
            nn.Conv2d(in_channels = input_channels,
                      out_channels = hidden_channels,
                      kernel_size = 1,
                      stride = 1,
                      padding = 0,
                      bias = bias),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace = True)
        )

        self.Depthwise = nn.Sequential(
            nn.Conv2d(in_channels = hidden_channels,
                      out_channels = hidden_channels,
                      kernel_size = 3,
                      stride = strides,
                      padding = padding,
                      bias = bias,
                      groups = hidden_channels),
            nn.BatchNorm2d(hidden_channels),
            nn.ReLU6(inplace = True)
        )

        self.convnet2 = nn.Sequential(
            nn.Conv2d(in_channels = hidden_channels,
                      out_channels = output_channels,
                      kernel_size = 1,
                      stride = 1,
                      padding = 0,
                      bias = bias),
            nn.BatchNorm2d(output_channels)
        )

        self.identity = nn.Conv2d(in_channels = input_channels,
                                  out_channels = output_channels,
                                  kernel_size = 1,
                                  stride = strides)

    def forward(self, x):
        y = self.convnet1(x)
        y = self.Depthwise(y)
        y = self.convnet2(y)
        if (self.strides == 1 and self.same_channels):
            y = x + y
            #identity = self.identity(x)
            #y = y + identity #y는 7x7

        return y


In [ ]:
class MobileNetV2(nn.Module):
    def __init__(self, input_channels=3, class_num = 100):
        super().__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = input_channels, out_channels = 32, kernel_size = 3, padding = 1, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace = True)
        )

        self.layer2 = nn.Sequential(
            InvertedBlock(input_channels = 32,
                          output_channels = 16,
                          strides = 1,
                          expansion = 1)
        )

        self.layer3 = nn.Sequential(
            InvertedBlock(input_channels = 16,
                          output_channels = 24,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 24,
                          output_channels = 24,
                          strides = 2,
                          expansion = 6)
        )

        self.layer4 = nn.Sequential(
            InvertedBlock(input_channels = 24,
                          output_channels = 32,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 32,
                          output_channels = 32,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 32,
                          output_channels = 32,
                          strides = 2,
                          expansion = 6)
        )

        self.layer5 = nn.Sequential(
            InvertedBlock(input_channels = 32,
                          output_channels = 64,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 64,
                          output_channels = 64,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 64,
                          output_channels = 64,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 64,
                          output_channels = 64,
                          strides = 2,
                          expansion = 6)
        )

        self.layer6 = nn.Sequential(
            InvertedBlock(input_channels = 64,
                          output_channels = 96,
                          strides = 1,
                          expansion = 6),
            InvertedBlock(input_channels = 96,
                          output_channels = 96,
                          strides = 1,
                          expansion = 6),
            InvertedBlock(input_channels = 96,
                          output_channels = 96,
                          strides = 1,
                          expansion = 6)
        )

        self.layer7 = nn.Sequential(
            InvertedBlock(input_channels = 96,
                          output_channels = 160,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 160,
                          output_channels = 160,
                          strides = 2,
                          expansion = 6),
            InvertedBlock(input_channels = 160,
                          output_channels = 160,
                          strides = 2,
                          expansion = 6)
        )

        self.layer8 = nn.Sequential(
            InvertedBlock(input_channels = 160,
                          output_channels = 320,
                          strides = 1,
                          expansion = 6)
        )

        self.layer9 = nn.Sequential(
            nn.Conv2d(in_channels = 320,
                      out_channels = 1280,
                      kernel_size = 1),
            nn.AdaptiveAvgPool2d(1)
        )

        self.fc = nn.Linear(1280, class_num)


    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.layer8(x)
        x = self.layer9(x).view(x.size(0), -1)
        x = self.fc(x)

        return x


In [ ]:
def train(model, epochs, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    best_score = 10000
    best_model = None

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.to(device)

            optimizer.zero_grad()

            model_pred = model(img)

            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
        val_loss = validation(model, criterion, test_loader, device)

        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step()

        if best_score > val_loss:
            best_model = model
            best_score = val_loss

    return best_model


In [ ]:
def validation(model, criterion, test_loader, device):
    model.eval()

    val_loss = []

    with torch.no_grad():
        for img, label in tqdm(iter(test_loader)):
            img, label = img.float().to(device), label.to(device)

            model_pred = model(img)

            loss = criterion(model_pred, label)

            val_loss.append(loss.item())

        return np.mean(val_loss)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
model = MobileNetV2(input_channels = 3, class_num = CLASS_NUM)
model.eval()
optimizer = torch.optim.Adam(params=model.parameters(), lr = LEARNING_RATE)
scheduler = None

infer_model = train(model, EPOCHS, optimizer = optimizer, train_loader = train_loader, test_loader = valid_loader, scheduler = scheduler, device = device)

  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

Epoch [1], Train Loss : [2.36602] Val Loss : [1.13488]


  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.16732] Val Loss : [0.18042]


  0%|          | 0/211 [00:00<?, ?it/s]

  0%|          | 0/53 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.13286] Val Loss : [0.02062]


In [ ]:
torch.save(infer_model.state_dict(), '/content/drive/MyDrive/Pytorch_Models/MobileNetV2.pth')

In [ ]:
test_df = get_dataframe('/content/fruit_classification/test/test', 'test')

In [ ]:
test_dataset = CustomDataset(test_df['path'].values, None, df_transformer)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()

    model_preds = []

    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()

    print('Done.')
    return model_preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/89 [00:00<?, ?it/s]

Done.


In [ ]:
preds = LabelEncoder.inverse_transform(preds)

In [ ]:
pred_list = []
for idx, i in enumerate(preds):
    pred_list.append([idx, i])

In [ ]:
submission = pd.DataFrame(pred_list, columns = ['id', 'label'])
submission.head(5)

,id,label
0,0,Grape Blue
1,1,Banana
2,2,Pomegranate
3,3,Apricot
4,4,Cactus fruit


In [ ]:
submission.to_csv('/content/drive/MyDrive/Pytorch_Inference_Result/MobileNetV2.csv', index=False)